# Vector interpolation depending on a vector field

In [ ]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
import sys, os
sys.path.append(os.path.relpath("..//..//..//src//python"))
import hiped as hp
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square
from copy import deepcopy
import time

## 1) Mesh and physical field $u$

In [ ]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
#mesh = Mesh(unit_cube.GenerateMesh(maxh=0.2))

fesU =  L2(mesh, order = 0)**2 # works only for lowest order
#fesU = H1(mesh, order = 1) # works only for lowest order

# definition of the vector physical field
u_ngs = GridFunction(fesU)
u_ngs.Set(CF((x,-x)))

Draw(u_ngs, mesh,  vectors={"grid_size" : 20, "offset" : 0.5 })

## 2) Interpolation space and variables $x$

In [ ]:
domain = hp.Domain(5) # 2D
#domain = hp.Domain("diamond3") # 3D

penal = hp.Penalization("ramp",2)

Id = CF(((1,0),(0,1)), dims = (2,2))

f1vngs = hp.VertexFunction("f1v", f = lambda u:  u, dfdu = lambda u : Id, 
                        dimInput= 2, dimOutput= 2, flagNGSolve= True)

f2vngs = hp.VertexFunction("f2v", f = lambda u: u + CF((2,3)), dfdu = lambda u : Id, 
                        dimInput= 2, dimOutput= 2, flagNGSolve= True)

f3vngs = hp.VertexFunction("f3v", f = lambda u: 3* u, dfdu = lambda u : 3*Id, 
                        dimInput= 2, dimOutput= 2, flagNGSolve= True)

f4vngs = hp.VertexFunction("f4v", f = lambda u: 4* u, dfdu = lambda u : 4*Id, 
                        dimInput= 2, dimOutput= 2, flagNGSolve= True)

f5vngs = hp.VertexFunction("f5v", f = lambda u: CF((2,3)), dfdu = lambda u : 0*Id, 
                        dimInput= 2, dimOutput= 2, flagNGSolve= True)

interp_ngs = hp.Interpolation(domain, [f1vngs,f2vngs,f3vngs,f4vngs,f5vngs], penalization = penal, label = "interpVec")

fesRho = L2(mesh)
np.random.seed(seed = 0)
rho0 = interp_ngs.setInitialVariable(typeInit = "rand", radius = 2, NGSpace=fesRho)
rho_ngs = interp_ngs.projection(rho0, deepcopyFlag = True)
plt.figure()
interp_ngs.plot(rho_ngs)
plt.show()

## 3) Evaluations

In [ ]:
w_ngs, dwdx_ngs = interp_ngs.evalBasisFunction(rho_ngs)
f_ngs = interp_ngs.eval(rho_ngs, u_ngs, w_ngs) 
dfdu_ngs = interp_ngs.evaldu(rho_ngs, u_ngs, w_ngs) 
dfdx_ngs = interp_ngs.evaldx(rho_ngs, u_ngs, w_ngs, dwdx_ngs) 

In [ ]:
# interpolated values
dimF = 1
Draw(f_ngs[dimF], mesh)

In [ ]:
# dfdu (for Newton Raphson)
dimF = 1
dimU = 1
Draw(dfdu_ngs[dimF,dimU], mesh)

In [ ]:
# dfdrho (for optimization)
dimF = 1 # component of F
dimX = 1 # derivative along the dimension "dim" of X
Draw(dfdx_ngs["interpVec"][dimX][dimF], mesh)

# Comparison with a pure Numpy approach

In [ ]:
Idnp, C = np.eye(2).reshape(2,2,1), np.array([2,3]).reshape(2,1,1)

f1vnp = hp.VertexFunction("f1v", f = lambda u:  u, dfdu = lambda u : Idnp*np.ones(u.shape), 
                        dimInput= 2, dimOutput= 2)
f2vnp = hp.VertexFunction("f2v", f = lambda u:  u + C, dfdu = lambda u : Idnp*np.ones(u.shape), 
                        dimInput= 2, dimOutput= 2)
f3vnp = hp.VertexFunction("f3v", f = lambda u: 3* u, dfdu = lambda u : 3*Idnp*np.ones(u.shape), 
                        dimInput= 2, dimOutput= 2)
f4vnp = hp.VertexFunction("f4v", f = lambda u: 4* u, dfdu = lambda u : 4*Idnp*np.ones(u.shape), 
                        dimInput= 2, dimOutput= 2)
f5vnp = hp.VertexFunction("f5v", f = lambda u: C*np.ones(u.shape), dfdu = lambda u : 0*Idnp*np.ones(u.shape), 
                        dimInput= 2, dimOutput= 2)

interp_np = hp.Interpolation(domain, [f1vnp,f2vnp,f3vnp,f4vnp,f5vnp], penalization = penal, label = "interpVec")

#u_np = (u_ngs.vec.FV().NumPy()[:].reshape(-1,2).T).reshape(2,1,-1) # wtf with reshape ???
u_np = u_ngs.vec.FV().NumPy()[:].reshape(2,1,-1) # works

rho_np = {'interpVec' : np.array([gf.vec.FV().NumPy()[:] for gf in rho_ngs['interpVec']]).T}

rho_np0 = deepcopy(rho_np) # 
rho_np = interp_np.projection(rho_np0.copy(), copyFlag = True) # troubles with projection in 3D

w_np, dwdx_np = interp_np.evalBasisFunction(rho_np)
f_np = interp_np.eval({'interpVec': None}, u_np, w_np) 
dfdu_np = interp_np.evaldu({'interpVec': None}, u_np, w_np) 
dfdx_np = interp_np.evaldx({'interpVec': None}, u_np, w_np, dwdx_np) 

In [ ]:
dimF = 1
# f relative difference
fngsnp = GridFunction(fesRho)
fngsnp.vec.FV().NumPy()[:] = f_np[dimF,0,:]
Draw(Norm((f_ngs[dimF]-fngsnp)/fngsnp), mesh)

In [ ]:
dimF = 0
dimU = 0
# dfdu relative difference
dfdungsnp = GridFunction(fesRho)
dfdungsnp.vec.FV().NumPy()[:] = dfdu_np[dimF,dimU,:]
Draw(Norm((dfdu_ngs[dimF,dimU]-dfdungsnp)/dfdungsnp), mesh)

In [ ]:
keyX = 'interpVec'
dimF = 1
dimX = 1
# dfdx relative difference
dfdx_npngs = GridFunction(fesRho)
dfdx_npngs.vec.FV().NumPy()[:] = dfdx_np[keyX][dimF,dimX,:]
Draw(Norm((dfdx_ngs[keyX][dimX][dimF]-dfdx_npngs)/dfdx_npngs), mesh)
# Relative difference quite high with 3D domains... (probably due to differences between Numpy and CoefficientFunction evaluations)